In [ ]:
!pip install chromadb transformers accelerate sentence-transformers PyMuPDF langchain

In [ ]:
import fitz  # PyMuPDF
import chromadb
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [ ]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
model_id = "HuggingFaceH4/zephyr-7b-alpha"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map="auto")
qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

...

model-00004-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    return "\n".join(page.get_text() for page in doc)

In [ ]:
def split_text(text, chunk_size=500, chunk_overlap=50):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_text(text)

In [ ]:
def build_pdf_qa_index(pdf_path, collection_name="pdf_qa"):
    text = extract_text_from_pdf(pdf_path)
    chunks = split_text(text)
    embeddings = embedding_model.encode(chunks).tolist()

    client = chromadb.Client()
    collection = client.get_or_create_collection(name=collection_name)

    if len(collection.get()["ids"]) == 0:
        for i, (chunk, emb) in enumerate(zip(chunks, embeddings)):
            collection.add(documents=[chunk], embeddings=[emb], ids=[f"chunk_{i}"])

    return collection

In [ ]:
def answer_question_with_zephyr(collection, question, top_k=3):
    query_emb = embedding_model.encode([question])[0].tolist()
    results = collection.query(query_embeddings=[query_emb], n_results=top_k)
    context = " ".join(results["documents"][0])

    prompt = f"<|system|>You are a helpful assistant.</s>\n<|user|>Context:\n{context}\n\nQuestion: {question}</s>\n<|assistant|>"
    output = qa_pipeline(prompt, max_new_tokens=256, do_sample=True, temperature=0.5)
    return output[0]["generated_text"].split("<|assistant|>")[-1].strip()

In [ ]:
pdf_path = "/content/offer letter (3).pdf"
collection = build_pdf_qa_index(pdf_path)

In [ ]:
while True:
  question = input("\nAsk a question (or type 'exit'): ")
  if question.lower() == "exit":
    break
  answer = answer_question_with_zephyr(collection, question)
  print("Answer:", answer)


Ask a question (or type 'exit'): How much is the food card?
Answer: The food card is eligible for a Food Card with an amount of INR 500/- being credited to this card per month.

Ask a question (or type 'exit'): exit
